In [13]:
%load_ext autoreload
%autoreload 2

import os
os.environ["EOTDL_API_URL"] = "http://localhost:8000/"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


When staging a dataset, the metadata is first staged.

Users can retreive assets individually afterwards.

In [14]:
from eotdl.datasets import retrieve_datasets

retrieve_datasets()

['EuroSAT-RGB-small']

In [15]:
from eotdl.datasets import stage_dataset

stage_dataset("EuroSAT-RGB-small", version=2, force=True)

'/home/juan/.cache/eotdl/datasets/EuroSAT-RGB-small'

In [16]:
os.listdir(os.path.expanduser('~/.cache/eotdl/datasets/EuroSAT-RGB-small'))

['catalog.v2.parquet']

In [17]:
import geopandas as gpd

gdf = gpd.read_parquet("~/.cache/eotdl/datasets/EuroSAT-RGB-small/catalog.v2.parquet")
gdf.head()

,type,stac_version,stac_extensions,datetime,id,bbox,geometry,assets,links,repository
0,Feature,1.0.0,[],2025-02-06 14:42:04.051302,test.txt,"{'xmax': 0.0, 'xmin': 0.0, 'ymax': 0.0, 'ymin'...",POLYGON EMPTY,{'asset': {'checksum': '3da541559918a808c2402b...,[],eotdl
1,Feature,1.0.0,[],2025-02-06 14:42:04.051459,README.md-781854,"{'xmax': 0.0, 'xmin': 0.0, 'ymax': 0.0, 'ymin'...",POLYGON EMPTY,{'asset': {'checksum': 'ca77bb578863ff62c1343b...,[],eotdl
2,Feature,1.0.0,[],2025-02-06 14:42:04.051546,Industrial/Industrial_1743.jpg,"{'xmax': 0.0, 'xmin': 0.0, 'ymax': 0.0, 'ymin'...",POLYGON EMPTY,{'asset': {'checksum': '582fb1e054885a609c1e25...,[],eotdl
3,Feature,1.0.0,[],2025-02-06 14:42:04.051608,Industrial/Industrial_1273.jpg,"{'xmax': 0.0, 'xmin': 0.0, 'ymax': 0.0, 'ymin'...",POLYGON EMPTY,{'asset': {'checksum': '2d267caf0ef060780fec89...,[],eotdl
4,Feature,1.0.0,[],2025-02-06 14:42:04.051665,Industrial/Industrial_1117.jpg,"{'xmax': 0.0, 'xmin': 0.0, 'ymax': 0.0, 'ymin'...",POLYGON EMPTY,{'asset': {'checksum': '0204dd4a3296ea8be3b388...,[],eotdl


In [23]:
gdf.assets[1]

{'asset': {'checksum': 'ca77bb578863ff62c1343bc8df6aadee9ee559ec',
  'href': 'http://localhost:8000/datasets/67a4a528bf61748ac9d2cea4/stage/README.md-781854',
  'timestamp': datetime.datetime(2025, 2, 6, 14, 42, 4, 51489)}}

In [19]:
from eotdl.datasets import stage_dataset_file

for _, row in gdf.sample(3).iterrows():
    for k, v in row['assets'].items():
        # print(asset['href'])
        path = stage_dataset_file(v['href'], "data/outputs")
        print(path)
	# stage_dataset_file(row["id"], row["assets"][0]["href"], user)


data/outputs/AnnualCrop/AnnualCrop_1142.jpg
data/outputs/PermanentCrop/PermanentCrop_1470.jpg
data/outputs/Forest/Forest_841.jpg


Or stage everything at once

In [26]:
!rm -rf data/outputs/*

In [27]:
from eotdl.datasets import stage_dataset

dst_path = stage_dataset("EuroSAT-RGB-small", path="data/outputs", assets=True, version=2, force=True)
dst_path

Staging assets: 100%|██████████| 102/102 [00:01<00:00, 91.88it/s]


'data/outputs/EuroSAT-RGB-small'

In [28]:
from glob import glob

files = glob(str(dst_path) + '/**/*.jpg', recursive=True)
len(files), files[:10]

(100,
 ['data/outputs/EuroSAT-RGB-small/Industrial/Industrial_1743.jpg',
  'data/outputs/EuroSAT-RGB-small/Industrial/Industrial_1273.jpg',
  'data/outputs/EuroSAT-RGB-small/Industrial/Industrial_1117.jpg',
  'data/outputs/EuroSAT-RGB-small/Industrial/Industrial_1121.jpg',
  'data/outputs/EuroSAT-RGB-small/Industrial/Industrial_1641.jpg',
  'data/outputs/EuroSAT-RGB-small/Industrial/Industrial_259.jpg',
  'data/outputs/EuroSAT-RGB-small/Industrial/Industrial_435.jpg',
  'data/outputs/EuroSAT-RGB-small/Industrial/Industrial_674.jpg',
  'data/outputs/EuroSAT-RGB-small/Industrial/Industrial_905.jpg',
  'data/outputs/EuroSAT-RGB-small/Industrial/Industrial_238.jpg'])

Staging virutal dataset

In [9]:
dst_path = stage_dataset("Test-links", force=True)
dst_path

'/home/juan/.cache/eotdl/datasets/Test-links'

In [10]:
gdf = gpd.read_parquet(f"{dst_path}/catalog.parquet")
gdf.head()

,stac_extensions,id,bbox,geometry,assets,links,collection,abc,123
0,[],https://link1.com,"{'xmax': 0.0, 'xmin': 0.0, 'ymax': 0.0, 'ymin'...",POLYGON EMPTY,[{'href': 'https://link1.com'}],[],Test-links,[],"{'asfhjk': [1, 2, 3]}"
1,[],https://link2.com,"{'xmax': 0.0, 'xmin': 0.0, 'ymax': 0.0, 'ymin'...",POLYGON EMPTY,[{'href': 'https://link2.com'}],[],Test-links,[],"{'asfhjk': [1, 2, 3]}"
2,[],https://link3.com,"{'xmax': 0.0, 'xmin': 0.0, 'ymax': 0.0, 'ymin'...",POLYGON EMPTY,[{'href': 'https://link3.com'}],[],Test-links,[],"{'asfhjk': [1, 2, 3]}"


In [11]:
dst_path = stage_dataset("Test-links", assets=True, force=True)
dst_path

Staging assets:   0%|          | 0/3 [00:00<?, ?it/s]

Staging assets: 100%|██████████| 3/3 [00:01<00:00,  1.93it/s]


'/home/juan/.cache/eotdl/datasets/Test-links'